#  NWB File generation from raw data

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import logging
import sys
from rec_to_nwb.processing.builder.nwb_file_builder import NWBFileBuilder
from rec_to_binaries import extract_trodes_rec_file
from rec_to_nwb.processing.builder.raw_to_nwb_builder import RawToNWBBuilder
from rec_to_nwb.processing.metadata.metadata_manager import MetadataManager


## Setup config parameters


In [6]:
# set the animal name and the date or list of dates to process
animal_name = 'beans'
dates = ['20190718']

yaml_path = '/Users/loren/Src/NWB/franklabnwb/yaml'
animal_metadata_file = 'beans20190718_metadata.yml'
probe1_metadata_file = '128c-4s8mm6cm-20um-40um-sl.yml'
probe2_metadata_file = 'tetrode_12.5.yml'

#Specify the paths for the data, the output nwb file, and the video files
data_path = '/Users/loren/data/nwb_builder_test_data/'
output_path='/Users/loren/data/nwb_builder_test_data/tmp'
video_path='/Users/loren/data/nwb_builder_test_data/video/'

# Specify any optional trodes export flags
trodes_rec_export_args = ('-reconfig', '/Users/loren/data/nwb_builder_test_data/beans/Probe_128ch_allnT_DIOs_PTP_reconfig_export_shanks.xml') 


# specify the locations of the metadata files for the animal and the probe(s). 
# Note that specifying all possible probes is fine
animal_metadata = os.path.join(yaml_path, animal_metadata_file )
probe_metadata = [os.path.join(yaml_path, probe1_metadata_file), 
                  os.path.join(yaml_path, probe2_metadata_file)]


# Specify whether data should be reextracted. 
overwrite=False

In [7]:
### Print the meta data as a check

In [8]:
metadata = MetadataManager(animal_metadata, probe_metadata)

print(metadata)


metadata path = /Users/loren/Src/NWB/franklabnwb/yaml/beans20190718_metadata.yml, probes_paths = ['/Users/loren/Src/NWB/franklabnwb/yaml/128c-4s8mm6cm-20um-40um-sl.yml', '/Users/loren/Src/NWB/franklabnwb/yaml/tetrode_12.5.yml']
Experiment Info:
Experimenter: Alison Comrie
Description: Reinforcement learning
Session Id: beans_01
Subject: Long Evans Rat

Available probe types: ['128c-4s8mm6cm-20um-40um-sl', 'tetrode_12.5']


# Processing initiation

In [9]:
builder = RawToNWBBuilder(animal_name=animal_name,
                          data_path=data_path,
                          dates=dates,
                          nwb_metadata=metadata,
                          overwrite=overwrite,
                          output_path=output_path,
                          video_path=video_path,
                          trodes_rec_export_args = trodes_rec_export_args)

## Run processing and create nwb file

In [ ]:
%%time
content = builder.build_nwb()
print(content)
#expected warnings from rec_to_binaries

/Users/loren/Src/NWB/rec_to_binaries/rec_to_binaries/trodes_data.py:551: UserWarning: Invalid folder name in preprocessing folder date (20190718) folder (test), ignoring.
  format(date, date_path_entry.name)))
/Users/loren/Src/NWB/rec_to_binaries/rec_to_binaries/trodes_data.py:1367: TrodesDataFormatWarning: TrodesDataFormatError('skipping rec file 20190718_beans_01_s1.rec for extracting, folder /Users/loren/data/nwb_builder_test_data/beans/preprocessing/20190718/20190718_beans_01_s1.analog already exists and overwrite=False.') (thrown from /Users/loren/Src/NWB/rec_to_binaries/rec_to_binaries/trodes_data.py:1308)
  TrodesDataFormatWarning)
/Users/loren/Src/NWB/rec_to_binaries/rec_to_binaries/trodes_data.py:1367: TrodesDataFormatWarning: TrodesDataFormatError('skipping rec file 20190718_beans_02_r1.rec for extracting, folder /Users/loren/data/nwb_builder_test_data/beans/preprocessing/20190718/20190718_beans_02_r1.analog already exists and overwrite=False.') (thrown from /Users/loren/Src/

[PosixPath('/Users/loren/data/nwb_builder_test_data/beans/raw/20190718/20190718_beans_01_s1.rec'), PosixPath('/Users/loren/data/nwb_builder_test_data/beans/raw/20190718/20190718_beans_03_s2.rec'), PosixPath('/Users/loren/data/nwb_builder_test_data/beans/raw/20190718/20190718_beans_02_r1.rec'), PosixPath('/Users/loren/data/nwb_builder_test_data/beans/raw/20190718/20190718_beans_04_r2.rec')]


# Cleanup preprocessed data
## Note that this deletes the extracted data, so you may wish to skip this step

In [ ]:
builder.clean_up()

## Test: Import the NWB file and inspect the contents

In [23]:
import pynwb
nwb_file_name = '/Users/loren/data/nwb_builder_test_data/beans20190718.nwb'
io = pynwb.NWBHDF5IO(nwb_file_name,'r')
nwbf = io.read()

In [24]:
nwbf

root pynwb.file.NWBFile at 0x140579284432464
Fields:
  acquisition: {
    e-series <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    SpikeGadgets <class 'ndx_franklab_novela.data_acq_device.DataAcqDevice'>,
    camera_device 0 <class 'ndx_franklab_novela.camera_device.CameraDevice'>,
    camera_device 1 <class 'ndx_franklab_novela.camera_device.CameraDevice'>,
    header_device <class 'ndx_franklab_novela.header_device.HeaderDevice'>,
    probe 0 <class 'ndx_franklab_novela.probe.Probe'>
  }
  electrode_groups: {
    0 <class 'ndx_franklab_novela.nwb_electrode_group.NwbElectrodeGroup'>,
    1 <class 'ndx_franklab_novela.nwb_electrode_group.NwbElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  epochs: epochs <class 'pynwb.epoch.TimeIntervals'>
  experiment_description: Reinforcement learning
  experimenter: ['Alison Comrie']
  file_create_date: [datetime.datetime(2020, 7, 9, 8, 34, 46, 756427, tzinfo=tzoffset(None, -25200))]
  identifier

In [5]:
lst = [12] 
",".join([str(i) for i in lst])

'12'